<a href="https://colab.research.google.com/github/madhavamk/computational-data-science/blob/master/MiniProjects/M6_NB_MiniProject_1_Image_Classification_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Mini Project Notebook: Image Classification using Multi Layer Perceptron

## Learning Objectives

At the end of the experiment, you will be able to :

* load and extract features of images

* implement the Multi-Layer perceptron to classify images

* implement simple neural network from keras

## Introduction

Traffic sign recognition is a challenging, real-world problem relevant for AI based transportation systems. Traffic signs show a wide range of variations between classes in terms of color, shape, and the presence of pictograms or text. However, there exist subsets of
classes (e.g., speed limit signs) that are very similar to each other. Further, the classifier
has to be robust against large variations in visual appearances due to changes in illumination, partial
occlusions, rotations, weather conditions etc. Using a comprehensive traffic sign detection dataset, here we will perform classification of traffic signs, train and evaluate the different models and compare to the performance of MLPs.

![img](https://paperswithcode.com/media/datasets/GTSRB-0000000633-9ce3c5f6_Dki5Rsf.jpg)

## Dataset

The data for this mini-project is from the German Traffic Sign Detection Benchmark [GTSDB](https://benchmark.ini.rub.de/gtsdb_dataset.html). This archive contains the training set used during the IJCNN 2013 competition.

The German Traffic Sign Detection Benchmark is a single-image detection assessment for researchers with interest in the field of computer vision, pattern recognition and image-based driver assistance. It is introduced on the IEEE International Joint Conference on Neural Networks 2013.

It features ...

* The main archive FullIJCNN2013.zip includes the images (1360 x 800 pixels) in PPM format, the image sections containing only the traffic signs
* A file in CSV format with the ground truth
* A ReadMe.txt with more details.

Note that we will be using the images inside the image sections subfolders, containing only the traffic signs.

## Problem Statement

To build and improve upon a machine learning model for the classification of images and achieve a high accuracy final model.

## Grading = 10 Points

In [1]:
#@title Download the data
!wget -qq https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
!unzip -qq FullIJCNN2013.zip

### Import Required packages

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage.io import imread, imshow
from sklearn import preprocessing
import os, glob
from PIL import Image
from sklearn.model_selection import GridSearchCV
# Keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

Data Loading and Feature Extraction (2 points)

#### Get the features and labels of data

* Extract the features of the images within image sections only (do not use images located outside these folders)
* Extract labels of the images
* Resize the images to (30, 30) and convert to numpy 1-D array

   Hint: [Link](https://machinelearningmastery.com/how-to-load-and-manipulate-images-for-deep-learning-in-python-with-pil-pillow/)

In [11]:
classes = {0:'speed limit 20 (prohibitory)',
1:'speed limit 30 (prohibitory)',
2:'speed limit 50 (prohibitory)',
3:'speed limit 60 (prohibitory)',
4:'speed limit 70 (prohibitory)',
5:'speed limit 80 (prohibitory)',
6:'restriction ends 80 (other)',
7:'speed limit 100 (prohibitory)',
8:'speed limit 120 (prohibitory)',
9:'no overtaking (prohibitory)',
10:'no overtaking (trucks) (prohibitory)',
11:'priority at next intersection (danger)',
12:'priority road (other)',
13:'give way (other)',
14:'stop (other)',
15:'no traffic both ways (prohibitory)',
16:'no trucks (prohibitory)',
17:'no entry (other)',
18:'danger (danger)',
19:'bend left (danger)',
20:'bend right (danger)',
21:'bend (danger)',
22:'uneven road (danger)',
23:'slippery road (danger)',
24:'road narrows (danger)',
25:'construction (danger)',
26:'traffic signal (danger)',
27:'pedestrian crossing (danger)',
28:'school crossing (danger)',
29:'cycles crossing (danger)',
30:'snow (danger)',
31:'animals (danger)',
32:'restriction ends (other)',
33:'go right (mandatory)',
34:'go left (mandatory)',
35:'go straight (mandatory)',
36:'go right or straight (mandatory)',
37:'go left or straight (mandatory)',
38:'keep right (mandatory)',
39:'keep left (mandatory)',
40:'roundabout (mandatory)',
41:'restriction ends (overtaking) (other)',
42:'restriction ends (overtaking (trucks)) (other)'}

In [12]:
# YOUR CODE HERE
def load_data_extract_features(data_path):
    img = []
    labels = []
    with open(data_path, 'r') as f:
        gtlines = f.readlines()
        for line in gtlines:
            file_details = line.split(';')
            image_name = file_details[0]
            label = int(file_details[-1].strip())
            img_path = os.path.join(os.getcwd(), 'FullIJCNN2013', image_name)
            image = Image.open(img_path)
            image = image.resize((30, 30))
            image = np.array(image)
            image = image.flatten()
            img.append(image)
            labels.append(label)
    return img, labels

img, labels = load_data_extract_features('FullIJCNN2013/gt.txt')
print(img)
print(labels)

[array([255, 255, 255, ...,  64,  70,  77], dtype=uint8), array([61, 63, 51, ..., 98, 96, 94], dtype=uint8), array([61, 63, 51, ..., 98, 96, 94], dtype=uint8), array([61, 63, 51, ..., 98, 96, 94], dtype=uint8), array([255, 255, 255, ..., 117, 122, 120], dtype=uint8), array([200, 204, 204, ..., 139, 129, 112], dtype=uint8), array([200, 204, 204, ..., 139, 129, 112], dtype=uint8), array([200, 204, 204, ..., 139, 129, 112], dtype=uint8), array([220, 221, 206, ...,  84,  87,  68], dtype=uint8), array([220, 221, 206, ...,  84,  87,  68], dtype=uint8), array([178, 196, 216, ...,  53,  56,  42], dtype=uint8), array([215, 255, 255, ...,  85,  86,  83], dtype=uint8), array([ 62,  64,  52, ..., 142, 136, 130], dtype=uint8), array([127, 159, 211, ..., 141, 128, 112], dtype=uint8), array([127, 159, 211, ..., 141, 128, 112], dtype=uint8), array([113, 141, 178, ..., 122,  95,  57], dtype=uint8), array([135, 175, 252, ..., 107,  97,  98], dtype=uint8), array([ 57,  66,  80, ..., 154, 139, 144], dtype

### Data Exploration and Preprocessing ( 2 points)

#### Plot the sample image of each class

Hint: plt.subplot

In [ ]:
# YOUR CODE HERE

#### Plot the distribution of Classes

In [ ]:
# YOUR CODE HERE

#### Normalize the features

For most image data, the pixel values are integers with values between 0 and 255.

Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. As such it is good practice to normalize the pixel values.

Hint: sklearn.preprocessing.normalize

In [ ]:
# YOUR CODE HERE

### Train the MLP classifier on features (1 point)

* Split the data into train and test

* Train the MLP classifier with different parameters

* Get the accuracy score and performance metrics

In [ ]:
# YOUR CODE HERE

### Tune the hyper-parameters (2 points)

* Use the GridSearchCV or RandomizedSearchCV and select best parameters

  Hint: [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

  (or)
* Manually change and find the best parameters

To know about all the parameters, click [here](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [ ]:
# YOUR CODE HERE

#### Try the different algorithms and compare the results with MLP classifier

In [ ]:
# YOUR CODE HERE

### Implement simple Neural Networks using keras (3 points)

* Define the keras model and initialize the layers
  - Ensure the input layer has the right number of input features. This can be specified when creating the first layer with the input_dim argument.
* Compile the model
  - Specify the loss function (to evaluate a set of weights), the optimizer (is used to search through different weights for the network) and any optional metrics to collect and report during training.
* Fit and Evaluate the model
  - Fit the data by specifying epochs and evaluate the model

In [ ]:
print(tf.__version__)

In [ ]:
# Step 1 - Build the architecture
# YOUR CODE HERE

In [ ]:
# Step 2 - Compile the model
# YOUR CODE HERE

In [ ]:
# Step 3 - Fit and Evaluate the model
# YOUR CODE HERE

#### Try the same parameters used for MLP Classifier and build the keras model

In [ ]:
# YOUR CODE HERE

#### Experiment using Dropout, Regularization and Batch Normalization

In [ ]:
# YOUR CODE HERE

### Report Analysis

* According to the confusion matrix, for which sign were the maximum misclassifications observed? Comment on the misclassification, owing to similar appearing traffic signs, if any.
* Comment on the performance of the MLP Classifier
* Discuss the optimal number of layers, activation functions, optimizers etc. that yielded the best accuracy
* Report on training time vs convergence

Reference: J. Stallkamp, M. Schlipsing, J. Salmen, and C. Igel. The German Traffic Sign Recognition Benchmark: A multi-class classification competition. In Proceedings of the IEEE International Joint Conference on Neural Networks, pages 1453–1460. 2011.